In [13]:
import pickle
import numpy as np
import neuraltoolkit as ntk
import glob
import csv
import os
import smart_open
import boto3
import tqdm
import argparse
import fnmatch
import io
import re
import tempfile
from pathlib import Path
from scripts.init import *
import pandas as pd

- Use this file to check the CNN results

First, initialize a client and obtain the prefix to the wanted files.

In [25]:
ANIMAL = ANIMALS_DATASET[-8]

client = get_s3_client()
summaries = get_animal_file(ANIMAL, client)

print(ANIMAL)
for summary in summaries:
    with smart_open.open(
        's3://hengenlab/'+summary, 'r', transport_params=dict(client=client)
    ) as f:
        print(f.read(375))

KDR36_day1
per-label-test-set (All Samples) (1295963 samples)
   0.91: WAKE            (accuracy) ( 694803 samples)
   0.85: NREM            (accuracy) ( 501821 samples)
   0.89: REM             (accuracy) (  99339 samples)
   0.89: F1 Multiclass  (avg=micro) (1295963 samples)
   0.86: F1 Multiclass  (avg=macro) (1295963 samples)
   0.89: Balanced Accuracy Score    (1295963 samples)

per-label-test-set (All Samples) (1295963 samples)
   0.91: WAKE            (accuracy) ( 694803 samples)
   0.87: NREM            (accuracy) ( 501821 samples)
   0.90: REM             (accuracy) (  99339 samples)
   0.89: F1 Multiclass  (avg=micro) (1295963 samples)
   0.87: F1 Multiclass  (avg=macro) (1295963 samples)
   0.89: Balanced Accuracy Score    (1295963 samples)

per-label-test-set (All Samples) (1295963 samples)
   0.91: WAKE            (accuracy) ( 694803 samples)
   0.86: NREM            (accuracy) ( 501821 samples)
   0.90: REM             (accuracy) (  99339 samples)
   0.89: F1 Multiclass  

In [16]:
print(f.read())


per-label-train-set (All Samples) (0 samples)
    NaN: WAKE            (accuracy) (      0 samples)
    NaN: NREM            (accuracy) (      0 samples)
    NaN: REM             (accuracy) (      0 samples)
   0.00: F1 Multiclass  (avg=micro) (      0 samples)
    NaN: F1 Multiclass  (avg=macro) (      0 samples)
    NaN: Balanced Accuracy Score    (      0 samples)

per-video-metrics (All Samples)
                                   Passive/   Micro-
    WAKE   |   NREM   |   REM    |  Active  |  arousal | video-filename
  -------- | -------- | -------- | -------- | -------- | ---------------
    0.73   |   0.29   |   0.30   |    NaN   |    NaN   | dummy (test-set)


per-label-test-set (Between State Change) (1187963 samples)
   0.73: WAKE            (accuracy) ( 970875 samples)
   0.29: NREM            (accuracy) ( 185238 samples)
   0.30: REM             (accuracy) (  31850 samples)
   0.65: F1 Multiclass  (avg=micro) (1187963 samples)
   0.39: F1 Multiclass  (avg=macro) (1187963 s

In [7]:
with smart_open.open(
    's3://hengenlab/CAF62_day1/Labels/labels_sleepstate_v2.1_CAF62_day1.npz',
    'rb',
    transport_params=dict(client=client)) as f:
    f1 = np.load(f)

In [8]:
f1.files

['labels_matrix', 'video_files', 'neural_files']

In [9]:
f1['video_files']

array(['dummy'], dtype='<U5')